In [ ]:
from lxml import etree
import json
import numpy as np

In [ ]:
def main():
    context = etree.iterparse("dblp.xml", dtd_validation=True)
    attribs=parse_features(context)
    with open('dblp_json.json','w') as json_file:
        json.dump(attribs,json_file)
    json_file.close()
        
def parse_features(root):
    types=[ 'www' , 'article', 'inproceedings', 'proceedings',\
           'incollection', 'phdthesis', 'mastersthesis']

    datadict = []
    for _,elem in root:
        if elem.tag in types:
            author=[]
            try:
                d={}
                for a in elem.findall('author'):
                    author.append(a.text)
                
                title= elem.find('title').text
                source= elem.find('booktitle').text
                year= elem.find('year').text
                for row in range(len(author)):
                    
                    d['Author']=author[row]
                    d['Coauthor']=author[:row]+author[row+1:]
                    d['Title']=title
                    d['booktitle']=source #change this to journal for records with journal tag
                    d['Year']=year
                    datadict.append(d)
                    d={}


            except:
                continue
    return datadict


if __name__=='__main__':
    main()

In [ ]:
# Insert the records into Mongo database
import pymongo
client = pymongo.MongoClient('localhost', 27017)
db=client.pymongo_test
coll=db["dblp_database"]

with open('dblp_json.json') as f:
    filedata=json.load(f)
    
coll.insert_many(filedata)  